In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("/content/sample_data/Time Series.xlsx")

In [ ]:
df.head()

,Name,Start,End,Activity
0,Priyanka,2023-10-10 22:43:52.620,2023-07-20 03:31:52.620,Inspection
1,Jyoti,2023-08-24 05:55:52.620,2023-05-17 20:19:52.620,Remote Inspection
2,Jyoti,2023-06-08 08:19:52.620,2023-04-08 05:55:52.620,Updates
3,Priyanka,2023-09-21 15:31:52.620,2023-05-27 10:43:52.620,Reporting
4,Priyanka,2023-10-07 03:31:52.620,2023-04-30 13:07:52.620,Reply to Customers


In [ ]:
df.shape

(10885, 4)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10885 entries, 0 to 10884
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Name      10885 non-null  object        
 1   Start     10885 non-null  datetime64[ns]
 2   End       10885 non-null  datetime64[ns]
 3   Activity  10885 non-null  object        
dtypes: datetime64[ns](2), object(2)
memory usage: 340.3+ KB


In [ ]:
df['Activity'].unique()

array(['Inspection', 'Remote Inspection', 'Updates', 'Reporting',
       'Reply to Customers', 'Fund raising', 'Business Development',
       'Send Email', 'Podcast', 'Call'], dtype=object)

In [ ]:
df.isnull().sum()

Name        0
Start       0
End         0
Activity    0
dtype: int64

In [ ]:
df = df.sort_values(by=['Name', 'Start'])

In [ ]:
df.head()

,Name,Start,End,Activity
4341,Deepti,2023-03-29 15:31:52.620,2023-04-02 08:19:52.620,Business Development
10807,Deepti,2023-03-29 15:31:52.620,2023-04-20 15:31:52.620,Reply to Customers
9894,Deepti,2023-03-29 17:55:52.620,2023-05-13 08:19:52.620,Send Email
10846,Deepti,2023-03-29 17:55:52.620,2023-03-30 22:43:52.620,Fund raising
181,Deepti,2023-03-29 20:19:52.620,2023-08-11 10:43:52.620,Send Email


In [ ]:
def find_continuous_periods(group):
    periods = []
    current_start = None
    current_end = None
    activities = []

    for _, row in group.iterrows():
        if current_start is None:
            current_start = row['Start']
            current_end = row['End']
            activities = [row['Activity']]
        elif row['Start'] <= current_end:
            current_end = max(current_end, row['End'])
            activities.append(row['Activity'])
        else:
            periods.append({
                'start': current_start,
                'end': current_end,
                'activities': activities
            })
            current_start = row['Start']
            current_end = row['End']
            activities = [row['Activity']]

    if current_start is not None:
        periods.append({
            'start': current_start,
            'end': current_end,
            'activities': activities
        })

    return pd.DataFrame(periods)

# Apply the function to each bot
result_df = df.groupby('Name').apply(find_continuous_periods).reset_index(drop=True)

# Save or display the result
result_df.to_excel('continuous_periods.xlsx', index=False)
print(result_df)


                     start                     end  \
0  2023-03-29 15:31:52.620 2023-10-15 15:31:52.620   
1  2023-03-29 15:31:52.620 2023-10-15 13:07:52.620   
2  2023-03-29 15:31:52.620 2023-10-15 15:31:52.620   
3  2023-03-29 15:31:52.620 2023-10-15 08:19:52.620   
4  2023-10-15 10:43:52.620 2023-06-10 17:55:52.620   
5  2023-10-15 10:43:52.620 2023-06-28 13:07:52.620   
6  2023-10-15 10:43:52.620 2023-07-26 01:07:52.620   
7  2023-10-15 10:43:52.620 2023-07-06 13:07:52.620   
8  2023-10-15 13:07:52.620 2023-04-18 10:43:52.620   
9  2023-10-15 13:07:52.620 2023-08-27 08:19:52.620   
10 2023-03-29 15:31:52.620 2023-10-15 05:55:52.620   
11 2023-10-15 08:19:52.620 2023-07-20 08:19:52.620   
12 2023-10-15 08:19:52.620 2023-08-14 17:55:52.620   
13 2023-10-15 08:19:52.620 2023-04-08 05:55:52.620   
14 2023-10-15 15:31:52.620 2023-07-23 17:55:52.620   

                                           activities  
0   [Business Development, Reply to Customers, Sen...  
1   [Business Developme

In [ ]:
result_df.head()

,start,end,activities
0,2023-03-29 15:31:52.620,2023-10-15 15:31:52.620,"[Business Development, Reply to Customers, Sen..."
1,2023-03-29 15:31:52.620,2023-10-15 13:07:52.620,"[Business Development, Inspection, Fund raisin..."
2,2023-03-29 15:31:52.620,2023-10-15 15:31:52.620,"[Business Development, Remote Inspection, Podc..."
3,2023-03-29 15:31:52.620,2023-10-15 08:19:52.620,"[Call, Call, Fund raising, Call, Fund raising,..."
4,2023-10-15 10:43:52.620,2023-06-10 17:55:52.620,[Send Email]


In [ ]:
result_df.shape

(15, 3)

In [ ]:
activities_df = result_df.explode('activities')

# Count occurrences of each activity
activity_counts = activities_df['activities'].value_counts()

print(activity_counts)

activities
Send Email              1158
Call                    1125
Updates                 1101
Fund raising            1098
Reply to Customers      1095
Inspection              1086
Remote Inspection       1076
Podcast                 1052
Business Development    1051
Reporting               1043
Name: count, dtype: int64


Using sql


In [ ]:
!pip install pandas openpyxl
import sqlite3

In [ ]:
data = pd.read_excel('/content/sample_data/Time Series.xlsx')

# Create a SQLite database and connect
conn = sqlite3.connect('work_log.db')
cursor = conn.cursor()

# Write the DataFrame to the SQLite database
df.to_sql('work_log', conn, if_exists='replace', index=False)

# Verify data has been loaded
query = "SELECT * FROM work_log LIMIT 5;"
pd.read_sql(query, conn)

,Name,Start,End,Activity
0,Deepti,2023-03-29 15:31:52.620000,2023-04-02 08:19:52.620000,Business Development
1,Deepti,2023-03-29 15:31:52.620000,2023-04-20 15:31:52.620000,Reply to Customers
2,Deepti,2023-03-29 17:55:52.620000,2023-05-13 08:19:52.620000,Send Email
3,Deepti,2023-03-29 17:55:52.620000,2023-03-30 22:43:52.620000,Fund raising
4,Deepti,2023-03-29 20:19:52.620000,2023-08-11 10:43:52.620000,Send Email


In [ ]:
query = '''
WITH OrderedWork AS (
    SELECT
        Name,
        Start,
        End,
        Activity,
        ROW_NUMBER() OVER (PARTITION BY name ORDER BY Start) AS rn
    FROM work_log
),
GroupedPeriods AS (
    SELECT
        Name,
        Start,
        End,
        Activity,
        rn - ROW_NUMBER() OVER (PARTITION BY name ORDER BY Start, End) AS grp
    FROM OrderedWork
),
ContinuousPeriods AS (
    SELECT
        Name,
        MIN(Start) AS period_start,
        MAX(End) AS period_end
    FROM GroupedPeriods
    GROUP BY Name, grp
)
SELECT
    cp.name,
    cp.period_start AS start,
    cp.period_end AS end,
    GROUP_CONCAT(w.activity, ', ') AS activities
FROM ContinuousPeriods cp
JOIN work_log w
    ON w.name = cp.name
    AND w.start <= cp.period_end
    AND w.end >= cp.period_start
GROUP BY cp.name, cp.period_start, cp.period_end
ORDER BY cp.name, cp.period_start;
'''

# Execute the SQL query
result_df = pd.read_sql(query, conn)

# Display the result
result_df

,Name,start,end,activities
0,Deepti,2023-03-29 15:31:52.620000,2023-10-15 15:31:52.620000,"Business Development, Reply to Customers, Send..."
1,Deepti,2023-03-29 17:55:52.620000,2023-10-06 10:43:52.620000,"Business Development, Reply to Customers, Send..."
2,Deepti,2023-03-29 17:55:52.620000,2023-10-15 05:55:52.620000,"Business Development, Reply to Customers, Send..."
3,Deepti,2023-03-31 17:55:52.620000,2023-08-30 20:19:52.620000,"Business Development, Reply to Customers, Send..."
4,Deepti,2023-04-05 08:19:52.620000,2023-10-14 20:19:52.620000,"Reply to Customers, Send Email, Send Email, Bu..."
5,Deepti,2023-04-17 15:31:52.620000,2023-07-05 20:19:52.620000,"Reply to Customers, Send Email, Send Email, Bu..."
6,Deepti,2023-04-21 13:07:52.620000,2023-05-24 03:31:52.620000,"Send Email, Send Email, Business Development, ..."
7,Deepti,2023-04-21 13:07:52.620000,2023-10-14 01:07:52.620000,"Send Email, Send Email, Business Development, ..."
8,Deepti,2023-05-06 10:43:52.620000,2023-04-13 20:19:52.620000,"Send Email, Send Email, Business Development, ..."
9,Deepti,2023-05-06 10:43:52.620000,2023-10-07 17:55:52.620000,"Send Email, Send Email, Business Development, ..."


In [ ]:
query = '''
WITH SortedWork AS (
    SELECT
        Name,
        Start,
        End,
        Activity
    FROM work_log
    ORDER BY Name, Start
),
OrderedWork AS (
    SELECT
        Name,
        Start,
        End,
        Activity,
        ROW_NUMBER() OVER (PARTITION BY Name ORDER BY Start) AS rn
    FROM SortedWork
),
GroupedPeriods AS (
    SELECT
        Name,
        Start,
        End,
        Activity,
        rn - ROW_NUMBER() OVER (PARTITION BY Name ORDER BY Start, End) AS grp
    FROM OrderedWork
),
ContinuousPeriods AS (
    SELECT
        Name,
        MIN(Start) AS period_start,
        MAX(End) AS period_end
    FROM GroupedPeriods
    GROUP BY Name, grp
)
SELECT
    cp.Name,
    cp.period_start AS start,
    cp.period_end AS end,
    GROUP_CONCAT(w.Activity, ', ') AS activities
FROM ContinuousPeriods cp
JOIN SortedWork w
    ON w.Name = cp.Name
    AND w.Start <= cp.period_end
    AND w.End >= cp.period_start
GROUP BY cp.Name, cp.period_start, cp.period_end
ORDER BY cp.Name, cp.period_start;
'''

# Execute the SQL query
result_df = pd.read_sql(query, conn)

# Display the result
result_df

,Name,start,end,activities
0,Deepti,2023-03-29 15:31:52.620000,2023-10-15 15:31:52.620000,"Business Development, Reply to Customers, Fund..."
1,Deepti,2023-03-29 17:55:52.620000,2023-10-06 10:43:52.620000,"Business Development, Reply to Customers, Fund..."
2,Deepti,2023-03-29 17:55:52.620000,2023-10-15 05:55:52.620000,"Business Development, Reply to Customers, Fund..."
3,Deepti,2023-03-31 17:55:52.620000,2023-08-30 20:19:52.620000,"Business Development, Reply to Customers, Send..."
4,Deepti,2023-04-05 08:19:52.620000,2023-10-14 20:19:52.620000,"Reply to Customers, Send Email, Business Devel..."
5,Deepti,2023-04-17 15:31:52.620000,2023-07-05 20:19:52.620000,"Reply to Customers, Send Email, Business Devel..."
6,Deepti,2023-04-21 13:07:52.620000,2023-05-24 03:31:52.620000,"Send Email, Business Development, Send Email, ..."
7,Deepti,2023-04-21 13:07:52.620000,2023-10-14 01:07:52.620000,"Send Email, Business Development, Send Email, ..."
8,Deepti,2023-05-06 10:43:52.620000,2023-04-13 20:19:52.620000,"Send Email, Business Development, Send Email, ..."
9,Deepti,2023-05-06 10:43:52.620000,2023-10-07 17:55:52.620000,"Send Email, Business Development, Send Email, ..."


In [ ]:
df = df.sort_values(by=['Name', 'Start'])

# Initialize variables
periods = []
current_start = None
current_end = None
activities = []
current_name = None

# Iterate through DataFrame rows
for _, row in df.iterrows():
    if current_name is None:
        # First row initialization
        current_name = row['Name']
        current_start = row['Start']
        current_end = row['End']
        activities = [row['Activity']]
    elif row['Name'] == current_name and row['Start'] <= current_end:
        # Overlapping period for the same Name
        current_end = max(current_end, row['End'])
        activities.append(row['Activity'])
    else:
        # Period ended or different Name
        periods.append({
            'Name': current_name,
            'start': current_start,
            'end': current_end,
            'activities': activities
        })
        # Reset for new period
        current_name = row['Name']
        current_start = row['Start']
        current_end = row['End']
        activities = [row['Activity']]

# Append the last period
if current_name is not None:
    periods.append({
        'Name': current_name,
        'start': current_start,
        'end': current_end,
        'activities': activities
    })

# Convert periods to DataFrame
result_df_python = pd.DataFrame(periods)

# Display the result
result_df_python

,Name,start,end,activities
0,Deepti,2023-03-29 15:31:52.620,2023-10-15 15:31:52.620,"[Business Development, Reply to Customers, Sen..."
1,Jyoti,2023-03-29 15:31:52.620,2023-10-15 13:07:52.620,"[Business Development, Inspection, Fund raisin..."
2,Priyanka,2023-03-29 15:31:52.620,2023-10-15 15:31:52.620,"[Business Development, Remote Inspection, Podc..."
3,Ravi,2023-03-29 15:31:52.620,2023-10-15 08:19:52.620,"[Call, Call, Fund raising, Call, Fund raising,..."
4,Ravi,2023-10-15 10:43:52.620,2023-06-10 17:55:52.620,[Send Email]
5,Ravi,2023-10-15 10:43:52.620,2023-06-28 13:07:52.620,[Reporting]
6,Ravi,2023-10-15 10:43:52.620,2023-07-26 01:07:52.620,[Inspection]
7,Ravi,2023-10-15 10:43:52.620,2023-07-06 13:07:52.620,[Inspection]
8,Ravi,2023-10-15 13:07:52.620,2023-04-18 10:43:52.620,[Send Email]
9,Ravi,2023-10-15 13:07:52.620,2023-08-27 08:19:52.620,[Podcast]


In [ ]:
periods = []
current_start = None
current_end = None
activities = []
current_name = None

# Iterate through DataFrame rows
for _, row in df.iterrows():
    if current_name is None:
        # First row initialization
        current_name = row['Name']
        current_start = row['Start']
        current_end = row['End']
        activities = [row['Activity']]
    elif row['Name'] == current_name and row['Start'] <= current_end:
        # Overlapping period for the same Name
        current_end = max(current_end, row['End'])
        activities.append(row['Activity'])
    else:
        # Period ended or different Name
        periods.append({
            'Name': current_name,
            'start': current_start,
            'end': current_end,
            'activities': ', '.join(set(activities))  # Aggregate activities
        })
        # Reset for new period
        current_name = row['Name']
        current_start = row['Start']
        current_end = row['End']
        activities = [row['Activity']]

# Append the last period
if current_name is not None:
    periods.append({
        'Name': current_name,
        'start': current_start,
        'end': current_end,
        'activities': ', '.join(set(activities))  # Aggregate activities
    })

# Convert periods to DataFrame
result_df_python = pd.DataFrame(periods)

# Display the result
result_df_python

,Name,start,end,activities
0,Deepti,2023-03-29 15:31:52.620,2023-10-15 15:31:52.620,"Reply to Customers, Inspection, Send Email, Po..."
1,Jyoti,2023-03-29 15:31:52.620,2023-10-15 13:07:52.620,"Inspection, Reply to Customers, Send Email, Po..."
2,Priyanka,2023-03-29 15:31:52.620,2023-10-15 15:31:52.620,"Inspection, Reply to Customers, Send Email, Po..."
3,Ravi,2023-03-29 15:31:52.620,2023-10-15 08:19:52.620,"Inspection, Reply to Customers, Send Email, Po..."
4,Ravi,2023-10-15 10:43:52.620,2023-06-10 17:55:52.620,Send Email
5,Ravi,2023-10-15 10:43:52.620,2023-06-28 13:07:52.620,Reporting
6,Ravi,2023-10-15 10:43:52.620,2023-07-26 01:07:52.620,Inspection
7,Ravi,2023-10-15 10:43:52.620,2023-07-06 13:07:52.620,Inspection
8,Ravi,2023-10-15 13:07:52.620,2023-04-18 10:43:52.620,Send Email
9,Ravi,2023-10-15 13:07:52.620,2023-08-27 08:19:52.620,Podcast
